In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
from osgeo import gdal, ogr
import numpy as np

In [ ]:
### open FMC netcdf file with xarray ###
#fmc_nc = xr.open_dataset('http://dapds00.nci.org.au/thredds/dodsC/ub8/au/FMC/c6/mosaics/fmc_c6_2020.nc')
fmc_nc = xr.open_dataset('/g/data/ub8/au/FMC/c6/mosaics/fmc_c6_2020.nc')
print(fmc_nc)

In [ ]:
### explore data in netcdf file ###
print(fmc_nc.coords) 
print(fmc_nc.variables)
print(fmc_nc.dims)
print(fmc_nc.lfmc_median)

In [ ]:
### filter by date ###
one_day = fmc_nc.sel(time='2020-01-01')
fmc_one_day = one_day.lfmc_median
fmc_one_day_array = fmc_one_day.data # this is now a 2d array

In [ ]:
### visualise array ###
plt.imshow(fmc_one_day_array, interpolation='none')
plt.show()

In [ ]:
### filter by date using indeces ###
fmc_one_day_idx = fmc_nc.isel(time=0).lfmc_median.data  # 0 is first index, in this case 1st Jan 2020  --- this is now a 2d array

In [ ]:
### filter by coords ###
one_point = fmc_nc.sel(latitude=-20.0075, longitude=140.985, method='nearest', tolerance=0.005) #tolerance in degrees
fmc_one_point = one_point.lfmc_median
fmc_one_point_time_series = fmc_one_point.data # this is now a 1d array

In [ ]:
### filter using range of coords and time ###
filtered_by_ranges = fmc_nc.sel(time=slice('2020-01-01','2020-01-31'), latitude=slice(-20,-30), longitude=slice(130,140)) # all dates in Jan 2020, latitude from -20 to -30, longitude from 130 to 140
fmc_filterd_by_ranges = filtered_by_ranges.lfmc_median.data # this is now a 3d array

In [ ]:
### compute mean across time dimension ###
monthly_mean = filtered_by_ranges.lfmc_median.mean(dim='time') # Nan are not considered in the computation of the mean

monthly_mean_array = monthly_mean.data 

In [ ]:
### visualisation ###
plt.imshow(monthly_mean, interpolation='none')
plt.show()

In [ ]:
### alternative way of filtering ###
filtered_by_idx = fmc_nc.sel(time='2020-01-01').lfmc_median.data[500:1000,6000:6500]  #[latitude, longitude]  --- this is now a 2d array


In [ ]:
### filter with shapefile ###
lfmc_array = fmc_nc.sel(time='2020-01-01').lfmc_median.data

# same dimensions used to create LFMC netcdf files
lat0 = -10.
lat1 = -44.
lon0 = 113.
lon1 = 154.

res = 0.005

x_size = int((lon1 - lon0)/res)
y_size = int((lat1 - lat0)/(-1*res))
lats = np.linspace(lat0, lat1+res, num=y_size)
lons = np.linspace(lon0, lon1-res, num=x_size)

geot = [lon0 - res/2, res, 0., lat0 + res/2, 0., -1*res] 

wgs84_wkt = 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.01745329251994328,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]]'

# create empty raster, same size of LFMC images
fire_modis_res = gdal.GetDriverByName('MEM').Create('', x_size, y_size, 1, gdal.GDT_Int16)
#fire_modis_res = gdal.GetDriverByName('GTiff').Create('PATH/orroral_raster.tif', x_size, y_size, 1, gdal.GDT_Int16)    # if want to save file
fire_modis_res.SetProjection(wgs84_wkt)
fire_modis_res.SetGeoTransform(geot)

fire_shp = ogr.Open(r'PATH\\orroral_fire.shp', 0)
fire_shp_layer = fire_shp.GetLayer(0)

# create raster mask of same size and res of LFMC MODIS array, the mask will have pixels equal to 1 where the polygon shapefile is located
# use 'ALL_TOUCHED=TRUE' if want to include all modis pixels that touch the polygon
# otherwise, only pixels whose centre is inside the polygon will be considered
gdal.RasterizeLayer(fire_modis_res, [1], fire_shp_layer, burn_values=[1], options=['ALL_TOUCHED=TRUE']) # (empty raster to imprint, raster band to use, sapefile to rasterize, values of the rasterize shapefile, ...) 

#fire_modis_res.FlushCache() #if chosen to save file

fire_event_mask = fire_modis_res.GetRasterBand(1).ReadAsArray() # mask to use with LFMC arrays

lfmc_fire_event = np.where(fire_event_mask==1, lfmc_array, np.nan)